## Visualising annotations

Generates PNG files

Author @ysbecca

In [24]:
import matplotlib.pyplot as plt
import urllib.request
from xml.dom import minidom
import sys

sys.path.insert(0, './scripts')

import fetch_samples as fs
import cv2
import numpy as np
import ysb
import csv

from PIL import Image

In [114]:
csv_path = "/Users/ysbecca/ysbecca-projects/bcsp-expert/data/test_cases_only.csv"
cases = []

with open(csv_path, 'r') as csv_file:
        reader = csv.reader(csv_file, delimiter=' ', quotechar='|')
        for row in reader:
                cases.append(row[0])

print("Found", len(cases), "cases.")

base_url = "http://129.11.191.7/Research_4/Slide_Library/NHS_BCSP/Experts_Board/Case_"
csv_path = "/Users/ysbecca/ysbecca-projects/bcsp-expert/test_annotations_roi/"
image_folder = "/Users/ysbecca/ysbecca-projects/bcsp-expert/saved_images/"

image_id_path = "/Users/ysbecca/ysbecca-projects/bcsp-expert/data/test_image_ids.csv"
image_ids = []
with open(image_id_path, 'r') as csv_file:
        reader = csv.reader(csv_file, delimiter=' ', quotechar='|')
        for row in reader:
                image_ids.append(row[0])

print("Found", len(image_ids), "image ids.")

paths = []
urls = []

for i, image_id in enumerate(image_ids):
    paths.append(csv_path + str(image_id) + ".xml")
    urls.append(base_url + cases[i] + "/" + str(image_id) + ".svs")

Found 50 cases.
Found 50 image ids.


In [115]:
# Checking for match-up
for i in range(len(paths)):
    if paths[0][65:-4] != urls[0][79:-4]:
        print(paths[i])
        print(urls[i])

In [116]:
image_thumbs = []
for url in urls:
    image_thumbs.append(fs.fetch_thumbnail(url))

In [117]:
scales, x_y = [], []
for i, image in enumerate(image_thumbs):
    thumb_y, thumb_x, _ = np.shape(image)
    x, y = fs.fetch_size(urls[i])
    x_y.append((x, y))
    scale_x = x / thumb_x
    scale_y = y / thumb_y
    scale = (scale_x + scale_y) / 2.0
    scales.append(scale)

In [118]:
len(image_thumbs)

50

Now parse the XML files.

In [95]:
image_coords = []
errors = []
error, error_ids = 0, []
for i, path in enumerate(paths):
    try:
        xml = minidom.parse(path)
    except:
        errors.append(path)
        error = 1
        
    if not error:
        regions = xml.getElementsByTagName("Region")

        all_coords = []

        for region in regions:
            # key = region.getElementsByTagName("Attribute")[0].attributes['Value'].value

            vertices = region.getElementsByTagName("Vertex")
            coords = np.zeros((len(vertices), 2))

            for i, vertex in enumerate(vertices):
                coords[i][0] = vertex.attributes['X'].value
                coords[i][1] = vertex.attributes['Y'].value
            all_coords.append(coords)
        image_coords.append(all_coords)
    else:
        error = 0 # Reset for next round
        error_ids.append(i)
        image_coords.append([])

In [99]:
print(errors, error_ids)
print(len(image_ids))
print(len(image_coords))
image_coords[25]

['/Users/ysbecca/ysbecca-projects/bcsp-expert/test_annotations_roi/211808.xml'] [25]
50
50


[]

In [119]:
ann_images = []
for i in range(len(image_ids)):
    annotated_image = np.copy(image_thumbs[i])
    print(i, ":", len(image_coords[i]))
    if len(image_coords[i]) > 0:
        for coords in image_coords[i]:
            annotated_image = fs.draw_boundary(annotated_image, x_y[i], coords, scales[i])
        ann_images.append(annotated_image)
    else:
        ann_images.append(image_thumbs[i])

0 : 3
1 : 9
2 : 1
3 : 2
4 : 4
5 : 7
6 : 2
7 : 2
8 : 5
9 : 10
10 : 3
11 : 4
12 : 3
13 : 7
14 : 1
15 : 3
16 : 4
17 : 4
18 : 3
19 : 4
20 : 4
21 : 4
22 : 1
23 : 4
24 : 5
25 : 0
26 : 4
27 : 2
28 : 6
29 : 4
30 : 3
31 : 3
32 : 2
33 : 6
34 : 1
35 : 5
36 : 3
37 : 4
38 : 6
39 : 5
40 : 1
41 : 4
42 : 7
43 : 4
44 : 4
45 : 3
46 : 8
47 : 7
48 : 15
49 : 19


In [120]:
len(ann_images)

50

In [122]:
for i in range(len(image_ids)):
#     plt.figure(figsize=(17,22))
#     plt.imshow(ann_image)
#     plt.axis("off")
#     plt.show()
#     plt.close()

    im = Image.fromarray(ann_images[i])
    im.save(image_folder + str(image_ids[i]) + "_annotated.png")